<h1>Usaira Shahbaz</h1>
<h1>Phase-3 Preprocessing</h1>


Removing 'review_comment_title' column

In [17]:
import pandas as pd

df = pd.read_csv("/content/FINAL.csv")

df.drop(columns=['review_comment_title'], inplace=True)


Checking and displaying missing values per column and total missing values

In [18]:
print("\n Missing values per column:\n")
missing_values = df.isnull().sum()
print(missing_values)

print("\n Total missing values in the dataset:", missing_values.sum())


 Missing values per column:

Unnamed: 0                           0
customer_city                        0
seller_city                        833
seller_state                       833
product_category_name_english     2567
price                              833
payment_value                        3
payment_type                         3
order_status                         0
order_purchase_timestamp             0
order_delivered_customer_date     3421
review_score                       997
review_comment_message           69062
review_creation_date               997
dtype: int64

 Total missing values in the dataset: 79549


Missing Values Handling

In [19]:
df.dropna(subset=[
    'seller_city',
    'seller_state',
    'product_category_name_english',
    'payment_value',
    'payment_type',
    'order_delivered_customer_date',
    'review_comment_message',
    'review_creation_date'
], inplace=True)

df['price'].fillna(df['price'].mean())
df['price'] = df['price'].fillna(round(df['price'].mean(), 2))


print("\n Missing values per column after cleanup:\n")
remaining_missing = df.isnull().sum()


print("\n Total remaining missing values in the dataset:", remaining_missing.sum())


 Missing values per column after cleanup:


 Total remaining missing values in the dataset: 0


Duplicate Values Checking

In [20]:
duplicate_rows = df[df.duplicated()]
print(f" Number of duplicate rows found: {len(duplicate_rows)}")


print(f"\n Final dataset shape: {df.shape}")


 Number of duplicate rows found: 0

 Final dataset shape: (47211, 14)


Detection of outliers using IQR

In [21]:
def detect_outliers(df):
    for col in df.select_dtypes(include=['float64', 'int64']).columns:
        Q1, Q3 = df[col].quantile([0.25, 0.75])
        IQR = Q3 - Q1
        lower, upper = Q1 - 1.5 * IQR, Q3 + 1.5 * IQR
        outliers = df[(df[col] < lower) | (df[col] > upper)]
        if not outliers.empty:
            print(f"{col}: {len(outliers)} outliers (Bounds: {lower:.2f}, {upper:.2f})")

detect_outliers(df)

price: 3553 outliers (Bounds: -105.38, 282.26)
payment_value: 3937 outliers (Bounds: -142.60, 410.98)


Filling outliers with mean

In [22]:
for col in df.select_dtypes(include=['float64', 'int64']).columns:
    Q1, Q3 = df[col].quantile([0.25, 0.75])
    IQR = Q3 - Q1
    lower, upper = Q1 - 1.5 * IQR, Q3 + 1.5 * IQR
    mean_val = round(df[col].mean(), 2)
    df.loc[(df[col] < lower) | (df[col] > upper), col] = mean_val.astype(df[col].dtype)


print("Outliers filled with mean")


Outliers filled with mean


In [23]:
df.to_csv('cleaned-data.csv', index=False)